<a href="https://colab.research.google.com/github/AnantChandra/google-colab/blob/main/rohlik_sales_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q kaggle

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"anantchandra","key":"0d4a00a1837c8993c7fc12089269e19e"}'}

In [4]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [15]:
!kaggle competitions download -c rohlik-sales-forecasting-challenge-v2

 85% 40.0M/47.3M [00:00<00:00, 111MB/s] 
100% 47.3M/47.3M [00:00<00:00, 95.2MB/s]


In [16]:
!unzip rohlik-sales-forecasting-challenge-v2.zip

Archive:  rohlik-sales-forecasting-challenge-v2.zip
  inflating: calendar.csv            
  inflating: inventory.csv           
  inflating: sales_test.csv          
  inflating: sales_train.csv         
  inflating: solution.csv            
  inflating: test_weights.csv        


In [22]:
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns

In [18]:
sales_train = pd.read_csv('sales_train.csv', parse_dates=['date'])
sales_test = pd.read_csv('sales_test.csv', parse_dates=['date'])
inventory = pd.read_csv('inventory.csv')
calendar = pd.read_csv('calendar.csv', parse_dates=['date'])
test_weights = pd.read_csv('test_weights.csv')
solution = pd.read_csv('solution.csv')

In [19]:
sales_train = pd.merge(sales_train, inventory, how='left', on =['unique_id','warehouse'])
sales_test = pd.merge(sales_test, inventory, how='left', on =['unique_id','warehouse'])

In [20]:
for df in [sales_train,sales_test]:
    df.set_index('date',inplace=True)

In [23]:
np.setdiff1d(sales_train.columns, sales_test.columns)

array(['availability', 'sales'], dtype=object)

In [24]:
sales_train.drop(['availability'], axis=1, inplace=True)
sales_train.sort_values(['date','warehouse'], inplace=True)

In [25]:
from scipy.stats import kurtosis

In [26]:
result = sales_train.reset_index().groupby(['warehouse']).agg(
    count = ('date','size'),
    first_date = ('date','min'),
    last_date = ('date','max'),
    date_difference=('date', lambda x: x.max() - x.min()),
    var_sales = ('sell_price_main','var'),
    mean_price = ('sell_price_main','mean'),
    skew_price = ('sell_price_main','skew'),
    max_price = ('sell_price_main','max'),
    kurtosis_sales = ('sell_price_main',kurtosis)
)

In [27]:
result

,count,first_date,last_date,date_difference,var_sales,mean_price,skew_price,max_price,kurtosis_sales
warehouse,,,,,,,,,
Brno_1,643637,2020-08-01,2024-06-02,1401 days,5108.657471,67.873869,2.826800,1015.51,11.636842
Budapest_1,574582,2020-08-01,2024-06-02,1401 days,822068.394919,918.184147,4.306405,21682.99,34.278188
Frankfurt_1,198937,2021-12-08,2024-06-02,907 days,10.047742,2.977863,6.456303,50.15,69.010019
Munich_1,259333,2021-05-20,2024-06-02,1109 days,7.316594,3.002412,4.059457,38.75,33.219987
Prague_1,780566,2020-08-01,2024-06-02,1401 days,4496.440265,65.941507,2.614916,1112.54,9.307276
Prague_2,770709,2020-08-01,2024-06-02,1401 days,4287.697391,65.624530,2.540218,994.18,8.578498
Prague_3,779655,2020-08-01,2024-06-02,1401 days,4970.689358,67.141527,2.560767,963.01,8.294052


In [28]:
result = sales_test.reset_index().groupby(['warehouse']).agg(
    count = ('date','size'),
    first_date = ('date','min'),
    last_date = ('date','max'),
    date_difference= ('date', lambda x: (x.max() - x.min()).days+1)
)

In [29]:
result

,count,first_date,last_date,date_difference
warehouse,,,,
Brno_1,7196,2024-06-03,2024-06-16,14
Budapest_1,6576,2024-06-03,2024-06-16,14
Frankfurt_1,3222,2024-06-03,2024-06-16,14
Munich_1,3907,2024-06-03,2024-06-16,14
Prague_1,8714,2024-06-03,2024-06-16,14
Prague_2,8655,2024-06-03,2024-06-16,14
Prague_3,8751,2024-06-03,2024-06-16,14


In [30]:
del result